In [ ]:
import numpy as np
import pandas as pd
import scipy as sp
import os 
from IPython.display import display, HTML
from os.path import isfile, join
import random
from collections import Counter

In [ ]:
COLLECTIONS = ['TREC3', 'TREC5', 'TREC6', 'TREC7', 'TREC8', 'TREC2001', 'R04', 'TB06', 'TB06M', 'WEB14']

onerun_systems = {'TREC3':
        ['xerox3', 'inq101', 'nyuir2', 'clarta', 'crnlea' , 'citri1', 'siems1', 'citya1', 'dortd2', 
         'westp1', 'lsia0mw2', 'eth002', 'pircs1', 'padre2', 'erima1', 'acqnt1', 'virtu1'],
        
          'TREC6':
        ['aiatA1', 'att97ac', 'anu6ash1', 'city6ad', 'csiro97a2', 'Cor6A2qtcs', 'DCU97snt','gerua3', 'gmu97au1', 
         'ibms97a', 'Mercure2', 'iss97s', 'LNaShort', 'mds601', 'jalbse','nsasg2', 'pirc7Ad', 'Brkly21', 'glair64', 
         'umcpa197', 'INQ401', 'ispa2', 'uwmt6a2','VrtyAH6a'],
    
          'TREC7':
        ['acsys7as', 'APL985SC', 'att98atdc', 'Brkly24', 'Cor7A1clt', 'dsir07a02', 'ETHAC0', 'FLab7atE', 'fsclt7a', 'gersh2', 
         'ibmg98b', 'ic98san3', 'iit98au2', 'INQ503', 'jalbse011', 'kslsV1', 'LIAshort2', 'LNaTitDesc7', 'mds98td', 'MerAdRbtd', 
         'nectitech', 'nttdata7At1', 'ok7am', 'umd98a1', 'uwmt7a0'],  
          
        'AH99':
        ['Mer8Adtd4', 'apl8ctd', 'kdd8sh16', 'ok8amxc', 'MITSLStd', 'uwmt8a2', 'nttd8ame','pir9Attd', 'ric8dpx', 'mds08a4', 'AntHoc1', 
         'acsys8amn', 'att99atde', 'weaver2','plt8ah2', 'Dm8NbnR', 'fub99td', 'Flab8atd2', 'GE8ATD3', 'ibmg99b', 'ibms99a','iit99au1', 
         'ic99dafb', 'Sab8A4', 'Scai8Adhoc', 'UB99SW', 'tno8d3', 'UniNET8St','umd99a1', 'INQ603', 'isa50', 'unc8al52', 'surfahi2', 'UT800', '1'],
         }
    

In [ ]:
'''
    shuffle with return
'''
def my_shuffle(array):
    random.shuffle(array)
    return array

'''
    builds the chunks of 5 sistems
'''
def get_chunk(systems): 
    mat = np.tile(systems, 5).reshape(5,(len(systems)))
    equaltotal=True
    attempts = 0
    while equaltotal:
        attempts +=1
        
        # shuffle
        for i in range(5):
            mat[i] = my_shuffle(  np.array(mat[i])  )
        
        # not 2 equal columns
        equal = 0
        for i in range(len(mat)): #generate pairs
            for j in range(i+1,len(mat)): 
                if np.array_equal(mat[i], mat[j]):           #compare rows        
                #if not(np.array_equal(mat[:,i],mat[:,j])):   #compare columns -- DON'T USE
                    equal += 1
        
        # unique elements in a column
        repeti = 0
        for z in range(len(systems)):
            if len(np.unique(mat[:,z])) < 5:
                repeti +=1
        
        if (equal == 0) and (repeti==0):
            #####
            aftercheck = 0
            mat.sort(axis=0)
            for i in range(len(mat)):
                for j in range(i+1,len(mat)): 
                    if np.array_equal(mat[i], mat[j]):           #compare rows  
                        aftercheck +=1
                    
            if aftercheck == 0:
            ####
                equaltotal = False
            
        # Borderline case -- NEVER happened, still provides a sound division in chunks
        if attempts ==1000000:
            for i in range(5):
                mat[i] = np.roll(mat[i], i)
            equaltotal = False
    
    return (np.transpose(mat), attempts )


In [ ]:
'''
    main
'''

df_campione = pd.DataFrame(columns=['collection', 'time' ,'dfcamp', 'attempts'])

for COLLECTION in COLLECTIONS:
    print(COLLECTION)
    CSV_TABLE =  '../../src/Tables/'+COLLECTION+'.csv'
    
    real_table = pd.read_csv(CSV_TABLE, sep=',', header=0, index_col=0)
    systems = onerun_systems[COLLECTION]
    
    for time in range(20):
        random.seed(time)
        print('{}, '.format(time+1), end='')
        lista, att = get_chunk(systems)
        
        df_campione.loc[len(df_campione)] = [COLLECTION, time, lista, att]        
        df_campione.to_pickle('../../pickles/trials_ONERUN.pickle')
